In [1]:
# Run if working locally
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import sqlite3
from sqlite3 import Error
import pickle
import os, sys
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, config.root_path)

from src.dataset.dataset import RawData
from src.dataset.wikisection_preprocessing import (
    tokenize,
    clean_sentence,
    preprocess_text_segmentation,
    format_data_for_db_insertion,
)
from src.dataset.utils import truncate_by_token
from db.dbv2 import Table, AugmentedTable, TrainTestTable
import pprint

from utils.metrics import windowdiff, pk

from src.bertkeywords.src.similarities import Embedding, Similarities
from src.bertkeywords.src.keywords import Keywords
from src.encoders.coherence import Coherence
from src.dataset.utils import flatten, dedupe_list, truncate_string

<IPython.core.display.Javascript object>

In [3]:
dataset_type = "city"
table = Table(dataset_type)
augmented_table = AugmentedTable(dataset_type)
train_test_table = TrainTestTable(dataset_type)

<IPython.core.display.Javascript object>

In [4]:
data = table.get_all()

text_data = [x[1] for x in data]
text_labels = [x[2] for x in data]

<IPython.core.display.Javascript object>

In [5]:
all_segments = table.get_all_segments()
text_segments = [[y[1] for y in x] for x in all_segments]
segments_labels = [[1 if i == 0 else 0 for i, y in enumerate(x)] for x in all_segments]

<IPython.core.display.Javascript object>

In [6]:
samples = 5
max_tokens = 400

for i, (segment, labels) in enumerate(
    zip(text_segments[:samples], segments_labels[:samples])
):
    for sentence, label in zip(segment, labels):
        # this is the training case. During inference, we will have no idea
        # when segments start and when they end.
        pass

<IPython.core.display.Javascript object>

In [7]:
text_labels[:25]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

<IPython.core.display.Javascript object>

In [54]:
# initialize the coherence library
max_words_per_step = 4
coherence = Coherence(max_words_per_step=max_words_per_step)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2023-04-21 23:13:42.323174: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
Some weights of the model checkpoint at /Users/amitmaraj/.cache/torch/sentence_transformers/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-04-21 23:13:45.391854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_r

<IPython.core.display.Javascript object>

In [57]:
def get_weighted_average(weighted_similarities, weights):
    return sum(weighted_similarities) / sum(weights)


# importance testing
def compare_coherent_words(
    coherence_map,
    keywords_current,
    suppress_errors=False,
    same_word_multiplier=2,  # if set to 1, don't amplify the same words found
    no_same_word_penalty=1,  # if set to 1, don't penalize for not finding the same word.
):
    word_comparisons = []
    weights = []
    for i, keywords in enumerate(coherence_map[::-1]):
        for word_tuple in keywords:
            word = word_tuple[0]
            for second_word_tuple in keywords_current:
                second_word = second_word_tuple[0]
                second_word_importance = second_word_tuple[1]

                try:
                    word_one_emb = word_tuple[2]
                    word_two_emb = second_word_tuple[2]

                    if same_word_multiplier > 1:
                        flattened_coherence_words_only = [
                            element[0]
                            for sublist in coherence_map
                            for element in sublist
                        ]

                        num_occurrences = flattened_coherence_words_only.count(
                            second_word
                        )

                        if num_occurrences > 0:
                            # amplify words that are found as duplicates in the coherence map
                            # if the word shows up 1 time, amplify the weight by 2 times
                            weighting_multiplier = flattened_coherence_words_only.count(
                                second_word
                            ) + (same_word_multiplier - 1)
                        else:
                            # no same word penalty
                            weighting_multiplier = (
                                1 / no_same_word_penalty
                            )  # reduce the importance of this word

                    else:
                        weighting_multiplier = 1  # set to 1 in case this is turned off.

                    # this weight is a recipricol function that will grow smaller the further the keywords are away
                    # we want to put more importance on the current words, so we apply twice as much weight.
                    if i == 0:
                        weight = (weighting_multiplier * 2) / (i + 1)
                    else:
                        weight = (weighting_multiplier * 1) / (i + 1)

                    # multiply the weighting factor by the importance of the second word
                    weight *= second_word_importance

                    word_comparisons.append(
                        (
                            word,
                            second_word,
                            weight
                            * coherence.embedding_lib.get_similarity(
                                word_one_emb, word_two_emb
                            ),
                        )
                    )
                    weights.append(weight)
                except AssertionError as e:
                    if not suppress_errors:
                        print(e, word, second_word)

    return word_comparisons, weights


# TODO: add weighted average: https://www.google.com/search?q=weighted+average&rlz=1C5CHFA_enCA1019CA1024&sxsrf=APwXEdcb6dhJ5L_mvWvrWr4AxQcxOFB01g:1681098698316&tbm=isch&source=iu&ictx=1&vet=1&fir=V-LTDKtCElo89M%252C2WVwd1NrPkHFOM%252C_%253BVGk_lj0HALhXQM%252C2WVwd1NrPkHFOM%252C_%253ByzfbB4i3SpPTFM%252C5e7an03wLAdfhM%252C_%253B47HYmoDH6WlThM%252CsRXbJWfpyOLEOM%252C_%253BOsB4jtfzenfuyM%252CHKcmLkpfJ3xWqM%252C_&usg=AI4_-kRmBXgUWAm_nR3vDsLT17TqM5AvSQ&sa=X&ved=2ahUKEwi6hvvVtJ7-AhXJkIkEHe4JCX4Q_h16BAgoEAE#imgrc=V-LTDKtCElo89M
def coherence_tester(
    text_data,
    text_labels,
    max_tokens=256,
    max_str_length=30,
    prediction_thresh=0.7,
    pruning=1,  # remove one sentence worth of keywords
    pruning_min=7,  # remove the first sentence in the coherence map once it grows passed 6
    dynamic_threshold=False,
    threshold_warmup=10,  # number of iterations before using dynamic threshold
    last_n_threshold=5,  # will only consider the last n thresholds for dynamic threshold
):
    coherence_map = []
    predictions = []
    thresholds = []
    for i, (row, label) in enumerate(zip(text_data, text_labels)):
        threshold = prediction_thresh
        if dynamic_threshold and (i + 1) > threshold_warmup:
            last_n_thresholds = thresholds[(0 - last_n_threshold) :]
            last_n_thresholds.sort()
            mid = len(last_n_thresholds) // 2
            threshold = (last_n_thresholds[mid] + last_n_thresholds[~mid]) / 2
            print(f"median threshold: {threshold}")
        # compare the current sentence to the previous one
        if i == 0:
            predictions.append((0, 0))
        else:
            print(f"Sample Number: {i}")
            prev_row = text_data[i - 1]

            row = truncate_by_token(row, max_tokens)
            prev_row = truncate_by_token(prev_row, max_tokens)

            cohesion, keywords_prev, keywords_current = coherence.get_coherence(
                [row, prev_row], coherence_threshold=0
            )

            # add the keywords to the coherence map
            coherence_map.append(cohesion)
            if pruning > 0 and len(coherence_map) >= pruning_min:
                print("pruning...", len(coherence_map))
                coherence_map = coherence_map[
                    pruning:
                ]  # remove the pruning amount from the beginning of the list
                print("done pruning...", len(coherence_map))

            # truncate the strings for printing
            truncated_row = truncate_string(row, max_str_length)
            truncated_prev_row = truncate_string(prev_row, max_str_length)
            print(
                f"Coherence Map: {[[x[0] for x in c] for c in coherence_map]}, KW Curr: {[x[0] for x in keywords_current]}"
            )

            # compute the word comparisons between the previous (with the coherence map)
            # and the current (possibly the first sentence in a new segment)
            word_comparisons_with_coherence, weights = compare_coherent_words(
                [*coherence_map, keywords_prev], keywords_current
            )

            similarities_with_coherence = [
                comparison[2] for comparison in word_comparisons_with_coherence
            ]
            avg_similarity_with_coherence = sum(similarities_with_coherence) / (
                len(similarities_with_coherence) or 1
            )
            weighted_avg_similarity_with_coherence = get_weighted_average(
                similarities_with_coherence, weights
            )
            print(f"weighted: {weighted_avg_similarity_with_coherence}")

            # if the two sentences are similar, create a cohesive prediction
            # otherwise, predict a new segment
            if weighted_avg_similarity_with_coherence > threshold:
                print(
                    f"Label: {label}, Prediction: {0}, logit: {weighted_avg_similarity_with_coherence}"
                )
                predictions.append((weighted_avg_similarity_with_coherence, 0))
            else:
                # start of a new segment, empty the map
                coherence_map = []
                print(
                    f"Label: {label}, Prediction: {1}, logit: {weighted_avg_similarity_with_coherence}"
                )
                predictions.append((weighted_avg_similarity_with_coherence, 1))

            thresholds.append(weighted_avg_similarity_with_coherence)
            print("===============================================")

    return predictions

<IPython.core.display.Javascript object>

In [100]:
start = 0
num_samples = 1500
max_tokens = 256  # want to keep this under 512
max_str_length = 30

true_labels = text_labels[start : start + num_samples]

predictions = coherence_tester(
    text_data[start : start + num_samples],
    true_labels,
    max_tokens=max_tokens,
    max_str_length=max_str_length,
)

Sample Number: 1
['sebastiáne', 'donesebastia', 'sebastián', 'sebastiane'] In spite of appearances both the Basque form Donostia and the Spanish form San Sebastián have the same meaning of Saint Sebastian The dona done doni element in Basque place names signifies saint and is derived from Latin domine the second part of Donostia contains shortened form of the saint name There are two hypotheses regarding the evolution of the Basque name one says it was Done Sebastiáne Donasa astiai Donasastia Donastia Donostia the other one says it was Done Sebastiane Done Sebastiae Done Sebastie Donesebastia Donasastia Donastia Donostia
['sebastián', 'surroundings', 'wetlands', 'picturesque'] The city is in the north of the Basque Autonomous Community on the southern coast of the Bay of Biscay San Sebastián picturesque shoreline makes it popular beach resort The seaside environment is enhanced by hilly surroundings that are easily accessible Urgull at the heart of the city by the seashore romantic Mou

['conquered', 'inhabiting', 'sebastián', 'pamplona'] After long period of silence in evidence in 1014 the monastery of St Sebastián with its apple orchards for cider located in the term of Hernani is donated to the Abbey of Leire by Sancho III of Pamplona By 1181 the city is chartered given fuero by king Sancho VI of Pamplona on the site of Izurum having jurisdiction over all the territory between the rivers Oria and Bidasoa In 1200 the city was conquered by Castile whose king Alfonso VIII confirmed its charter fuero but the Kingdom of Navarre was deprived of its main direct access out to the sea Perhaps as soon as 1204 or earlier the city nucleus at the foot of Urgull started to be populated with Gascon speaking colonizers from Bayonne and beyond who left an important imprint in the city identity in the centuries to come In 1265 the use of the city as seaport is granted to Navarre as part of wedding pact The large quantity of Gascons inhabiting the town favoured the development of tra

['sebastián', 'cristina', 'demolished', 'haussmannian'] As result of Donostia sprawling in all directions first into the flatlands shaped by the river Urumea and later up the hills new districts arose after the walls of the city were demolished in 1863 The first expansion of the old town stretched out to the river mouth on the old quarter called Zurriola name later given by Council decision to the sand area and the avenue across the river The orthogonal layout nowadays making up the city centre the Cortazar development was built up to 1914 first phase finished much in tune with Parisian Haussmannian style The arcades of the Buen Pastor square were fashioned after the ones of the Rue de Rivoli with the Maria Cristina Bridge being inspired by the Pont Alexandre III that spans the Seine The Estación del Norte train station standing right across the bridge was inaugurated in 1864 just after the arrival of the railway to San Sebastián with its metallic roof being designed by Gustave Eiffel 

['overshadowed', 'offices', 'sancho', 'nowadays'] This city expansion to the south came about as of the 1940s after the works to canalize the river were achieved Nowadays the name Amara usually applies to this sector the newer district having overshadowed the original nucleus both in size and population The district harbours the main road entrance to the city Facilities of many state run agencies were established here and presently Amara buildings house many business offices The district revolves around the axis of Avenida Sancho el Sabio and Avenida de Madrid
['proprietors', 'zurriola', 'demolished', 'chofre'] The district is built on the sandy terrain across the river The Gros or Zurriola surf beach by the river mouth bears witness to that type of soil In the 19th century shanties and workshops started to dot the area Tomas Gros being one of its main proprietors as well as providing the name for this part of the city The area held the former monumental bullring Chofre demolished in 1

['bidebieta', 'inhabitants', 'sebastián', 'trintxerpe'] Altza Basque for alder tree is the easternmost district of San Sebastián along with Bidebieta and Trintxerpe It was but quaint village comprising scattered farmhouses and small nucleus century ago 683 inhabitants in 1910 yet on the arrival of thousands of immigrants in the 1960s and 1970s rapid and chaotic housing and building activity ensued resulting in maze of grey landscape of skyscrapers and 32 531 inhabitants crammed in them data of 1970 the figure is 20 000 as of 2013
['sebastián', 'institutions', 'konporta', 'demolished'] Ibaeta stands on the former location for various factories Cervezas El León of San Sebastián with the buildings of the old industrial estate being demolished in the late 20th century The levelling of this large flat area paved the ground for carefully planned modern and elegant housing estate featuring new university campus for the public University of the Basque Country UPV EHU and institutions such as t

...

['households', 'poverty', 'km²', 'families'] As of the census of 2000 there were 128 people 47 households and 36 families residing in the city The population density was 689 people per square mile 260 km² There were 51 housing units at an average density of 274 per square mile 103 km² The racial makeup of the city was 97 66 White and 34 from two or more races There were 47 households out of which 42 had children under the age of 18 living with them 70 were married couples living together had female householder with no husband present and 23 were non families 19 of all households were made up of individuals and had someone living alone who was 65 years of age or older The average household size was 72 and the average family size was 17 In the city the population was spread out with 30 under the age of 18 from 18 to 24 32 from 25 to 44 21 from 45 to 64 and 12 who were 65 years of age or older The median age was 38 years For every 100 females there were 91 males For every 100 females age 

['tourist', 'stiniva', 'villages', 'viticulture'] The town of Vis is located in relatively large and protected bay English Bay of Saint George on the north east side of the island facing the island of Hvar and the Dalmatian mainland The port of Vis is located in the south west part of the bay The port is protected from open sea influence by the islet of Host named after William Hoste and the peninsula called Prirovo sometimes spelled Prilovo Other ports are smaller and located in Kut and Stonca which are also parts of the town of Vis Vis is isolated from its fields Dračevo polje and Velo polje thanks to 250 300 high hills which are important for the local citizens as the main source of incomes from viticulture Other official parts of the town on the south and south east coast are villages located on bays such as Milna Rukavac Srebrna Stiniva Stončica although some of them have tendency to become new towns These bays and villages are besides the town of Vis itself the main source of tou

['inhabited', 'provisional', 'kalapuya', 'anticipation'] Originally the area around Woodburn was inhabited by the Kalapuya Native Americans After the Provisional Government of Oregon set up land claims in the Oregon Country the United States annexed much of the Pacific Northwest and established the Oregon Territory in 1848 Congress passed the Donation Land Claim Act in 1850 and many earlier land claims became donation land claims Eli Cooley Bradford Bonney George Leisure and Jean Ducharme all established donation land claims on the eastern part of the French Prairie where Woodburn would later be founded Cooley immigrated to Oregon in 1845 and Bonney established his land claim in 1849 Ducharme land was sold off in 1862 in foreclosure with Mt Angel farmer George Settlemier purchase the on the cheap Settlemier had traveled west over the Oregon Trail in 1849 and first settled in California before moving north to Oregon in 1850 He settled in the Mt Angel area where he was successful nursery

['doukhobors', 'baptists', 'refugees', 'pentecostals'] Woodburn is home for sizable community of Russian Orthodox Old Believers This Christian traditionalist church had escaped persecution from the official Russian Orthodox Church and moved to the United States from Turkey in the 1950s Its women wear traditional long skirts and scarfs and men have beards Also present in the city are communities of Russian Molokans Doukhobors and recent refugees from the former USSR Ukrainian and Russian Pentecostals and Baptists
['institutions', 'businesses', 'economy', 'following'] The following businesses and institutions contribute to the economy of Woodburn
Got the keywords in 0.7648 seconds
Got the embeddings and comparisons in 0.0004 seconds
Coherence Map: [['institutions', 'businesses', 'economy', 'doukhobors']], KW Curr: ['doukhobors', 'baptists', 'refugees', 'pentecostals']
weighted: tensor([0.7761])
Label: 0, Prediction: 0, logit: tensor([0.7761])
Sample Number: 1498
['institutions', 'busines

<IPython.core.display.Javascript object>

In [101]:
print([x[1] for x in predictions])
print(true_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

<IPython.core.display.Javascript object>

In [102]:
pred_string = "".join(str([x[1] for x in predictions]))
true_string = "".join(str(true_labels))

<IPython.core.display.Javascript object>

In [103]:
avg_k = len(true_labels) // (true_labels.count(1) + 1)  # get avg segment size

<IPython.core.display.Javascript object>

In [104]:
wd_score = windowdiff(pred_string, true_string, avg_k)
pk_score = pk(pred_string, true_string, avg_k)

print(f"k = {avg_k}")
print(f"wd = {wd_score}")
print(f"pk = {pk_score}")

k = 6
wd = 0.41223581757508343
pk = 0.3835372636262514


<IPython.core.display.Javascript object>

## Prediction Tuning

In [105]:
pred_thresholds = [
    0.2,
    0.21,
    0.22,
    0.23,
    0.24,
    0.25,
    0.26,
    0.27,
    0.28,
    0.29,
    0.3,
]  # bert base uncased
pred_thresholds = [
    0.4,
    0.41,
    0.42,
    0.43,
    0.44,
    0.45,
    0.46,
    0.47,
    0.48,
    0.49,
    0.5,
]  # labse
pred_thresholds = [
    0.06,
    0.07,
    0.08,
    0.09,
    0.1,
    0.11,
    0.12,
    0.13,
    0.14,
    0.15,
    0.16,
    0.17,
    0.18,
    0.19,
    0.2,
    0.11,
    0.06,
]  # sentence-transformers
pred_thresholds = [
    0.6,
    0.61,
    0.62,
    0.63,
    0.64,
    0.65,
    0.66,
    0.67,
    0.68,
    0.69,
    0.7,
]  # USE
pred_thresholds = [
    0.65,
    0.66,
    0.67,
    0.68,
    0.69,
    0.7,
    0.71,
    0.72,
    0.73,
    0.74,
    0.75,
    0.76,
    0.77,
    0.78,
    0.79,
    0.64,
]  # Roberta

<IPython.core.display.Javascript object>

In [106]:
for pred_thresh in pred_thresholds:
    modified_predictions = [
        1 if x < pred_thresh else 0 for x in [x[0] for x in predictions]
    ]

    pred_string = "".join(str(modified_predictions))
    true_string = "".join(str(true_labels))

    avg_k = len(true_labels) // (true_labels.count(1) + 1)  # get avg segment size

    wd_score = windowdiff(pred_string, true_string, avg_k)
    pk_score = pk(pred_string, true_string, avg_k)

    print(f"pred_thresh = {pred_thresh}")
    print(f"k = {avg_k}")
    print(f"wd = {wd_score}")
    print(f"pk = {pk_score}")
    print("===========================================")

pred_thresh = 0.65
k = 6
wd = 0.3150166852057842
pk = 0.31167964404894327
pred_thresh = 0.66
k = 6
wd = 0.3256952169076752
pk = 0.3210233592880979
pred_thresh = 0.67
k = 6
wd = 0.33170189098998887
pk = 0.3270300333704116
pred_thresh = 0.68
k = 6
wd = 0.34505005561735264
pk = 0.33770856507230257
pred_thresh = 0.69
k = 6
wd = 0.36907675194660733
pk = 0.35372636262513907
pred_thresh = 0.7
k = 6
wd = 0.4117908787541713
pk = 0.3830923248053393
pred_thresh = 0.71
k = 6
wd = 0.4424916573971079
pk = 0.40378197997775306
pred_thresh = 0.72
k = 6
wd = 0.47986651835372635
pk = 0.4351501668520578
pred_thresh = 0.73
k = 6
wd = 0.5145717463848721
pk = 0.45917686318131257
pred_thresh = 0.74
k = 6
wd = 0.5612903225806452
pk = 0.4872080088987764
pred_thresh = 0.75
k = 6
wd = 0.6026696329254727
pk = 0.5152391546162403
pred_thresh = 0.76
k = 6
wd = 0.6484983314794216
pk = 0.5459399332591769
pred_thresh = 0.77
k = 6
wd = 0.6952169076751946
pk = 0.575305895439377
pred_thresh = 0.78
k = 6
wd = 0.751946607341

<IPython.core.display.Javascript object>

In [107]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

print(pred_string)
print(true_string)

tn, fp, fn, tp = confusion_matrix(true_labels, modified_predictions).ravel()
precision, recall, f1, _ = precision_recall_fscore_support(
    true_labels, modified_predictions, average="macro"
)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

<IPython.core.display.Javascript object>

In [108]:
wd_score = windowdiff(pred_string, true_string, avg_k)
pk_score = pk(pred_string, true_string, avg_k)

print(f"k = {avg_k}")
print(f"wd = {wd_score}")
print(f"pk = {pk_score}")
print(f"tn = {tn}")
print(f"fp = {fp}")
print(f"fn = {fn}")
print(f"tp = {tp}")
print(f"precision = {precision}")
print(f"recall = {recall}")
print(f"f1 = {f1}")

k = 6
wd = 0.3076751946607342
pk = 0.30567296996662957
tn = 1203
fp = 55
fn = 215
tp = 27
precision = 0.5888231449310262
recall = 0.533925028577435
f1 = 0.5328849028400597


<IPython.core.display.Javascript object>

## KeyBERT Embedding Comparison

In [172]:
curr = 230
prev = curr - 1

<IPython.core.display.Javascript object>

In [ ]:
# initialize the keywords and embeddings library
pp = pprint.PrettyPrinter(indent=4)
similarities_lib = Similarities("bert-base-uncased")
keywords_lib = Keywords(similarities_lib.model, similarities_lib.tokenizer)
embedding_lib = Embedding(similarities_lib.model, similarities_lib.tokenizer)

In [205]:
cohesion = coherence.get_coherence(
    [text_data[curr], text_data[prev]], coherence_threshold=0.25
)
print([k[0] for k in cohesion])

Got the keywords in 0.6567 seconds
Got the embeddings and comparisons in 0.0007 seconds
['cantonese', 'languages', 'vietnamese', 'communes']


<IPython.core.display.Javascript object>

In [206]:
# get the keywords for the current sentences
keywords_current = keywords_lib.get_keywords_with_kb_embeddings(text_data[curr])
keywords_prev = keywords_lib.get_keywords_with_kb_embeddings(text_data[prev])

# compute the word comparisons between the previous (with the coherence map)
# and the current (possibly the first sentence in a new segment)
word_comparisons_with_coherence, weights = compare_coherent_words(
    [keywords_prev], keywords_current
)

<IPython.core.display.Javascript object>

In [207]:
[(x[0], x[1]) for x in keywords_current], [(x[0], x[1]) for x in keywords_prev]

([('township', 0.2304),
  ('communes', 0.1857),
  ('hải', 0.1399),
  ('wards', 0.1397),
  ('đông', 0.1224)],
 [('cantonese', 0.5038),
  ('mandarin', 0.464),
  ('languages', 0.3483),
  ('language', 0.343),
  ('vietnamese', 0.3184)])

<IPython.core.display.Javascript object>

# KeyBERT Embedding Testing

In [679]:
docs = [
    "Hi my name is Devarsh",
    "Devarsh likes to play Basketball.",
    "I love to watch Cricket.",
    "I am a strong programmer. And my name is Devarsh",
]

<IPython.core.display.Javascript object>

In [680]:
from keybert import KeyBERT

kw_model = KeyBERT()
doc_embeddings, word_embeddings = kw_model.extract_embeddings(
    docs, min_df=1, stop_words="english"
)
keywords = kw_model.extract_keywords(
    docs,
    min_df=1,
    stop_words="english",
    doc_embeddings=doc_embeddings,
    word_embeddings=word_embeddings,
)

<IPython.core.display.Javascript object>

In [681]:
len(doc_embeddings)

4

<IPython.core.display.Javascript object>

In [682]:
len(word_embeddings)

10

<IPython.core.display.Javascript object>

In [683]:
keywords

[[('devarsh', 0.6267), ('hi', 0.5216)],
 [('devarsh', 0.6549),
  ('basketball', 0.5558),
  ('play', 0.3787),
  ('likes', 0.2284)],
 [('cricket', 0.7118), ('watch', 0.3656), ('love', 0.307)],
 [('programmer', 0.5942), ('devarsh', 0.5528), ('strong', 0.3452)]]

<IPython.core.display.Javascript object>

In [701]:
kw_model = KeyBERT()
import torch


def get_keywords_with_embeddings_test(
    data,
) -> list[tuple[str, float, torch.Tensor]]:
    doc_embeddings, word_embeddings = kw_model.extract_embeddings(data)

    keywords = kw_model.extract_keywords(
        data, doc_embeddings=doc_embeddings, word_embeddings=word_embeddings
    )

    keywords_with_embeddings = []
    count = 0
    print(len(word_embeddings))
    for i, (kw, we) in enumerate(zip(keywords, word_embeddings)):
        for j, words in enumerate(kw):
            keywords_with_embeddings.append((words[0], words[1], torch.tensor(we)))
            count += 1

    return keywords_with_embeddings

<IPython.core.display.Javascript object>

In [702]:
embeddings = get_keywords_with_embeddings_test(docs)

10


<IPython.core.display.Javascript object>

In [703]:
len(embeddings)

12

<IPython.core.display.Javascript object>